In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.linalg import solve_triangular as sp_solve_triangular

import jlinops
from jlinops import _CustomLinearOperator, get_device

In [20]:
# Set the size of the matrix
n = 5

# Generate a random n x n matrix
random_matrix = np.random.rand(n, n)

# Extract the upper triangular part of the matrix
Umat = np.triu(random_matrix)
U = jlinops.MatrixLinearOperator(Umat)
Uminv = np.linalg.inv(Umat)

In [21]:
jlinops.black_box_to_dense(U)

array([[0.98002196, 0.54286389, 0.98269779, 0.13235562, 0.35842242],
       [0.        , 0.53733683, 0.1145083 , 0.62932704, 0.58092173],
       [0.        , 0.        , 0.15377439, 0.8814042 , 0.59511893],
       [0.        , 0.        , 0.        , 0.50140539, 0.41794074],
       [0.        , 0.        , 0.        , 0.        , 0.59089279]])

In [22]:
class UpperTriangularInverse(_CustomLinearOperator):
    """Represents the inverse of an upper triangular dense matrix.

    U: upper triangular dense array U (lower triangular part will be discarded).
    """

    def __init__(self, U):

        device = get_device(U)
        self.U = np.triu(U.A)
        m = U.shape[0]
        n = U.shape[1]
        assert m == n, "U must be square."
        self.n = n

        def _matvec(x):
            return sp_solve_triangular(self.U, x, trans=0)

        def _rmatvec(x):

            return sp_solve_triangular(self.U, x, trans=1)
        
        super().__init__( (self.n, self.n), _matvec, _rmatvec, dtype=None, device=device)

    def to_gpu(self):
        raise NotImplementedError
    

In [23]:
Uinv = jlinops.UpperTriangularInverse(U)

In [24]:
np.linalg.norm( jlinops.black_box_to_dense( Uinv ) - Uminv )

2.1211235096247982e-15

In [25]:
np.linalg.norm( jlinops.black_box_to_dense( Uinv.T ) - Uminv.T )

5.149762648337517e-16

# Lower triangular

In [8]:
class LowerTriangularInverse(_CustomLinearOperator):
    """Represents the inverse of an lower triangular dense matrix.

    L: upper triangular dense array L (upper triangular part will be discarded).
    """

    def __init__(self, L):

        device = get_device(L)
        self.L = np.tril(L.A)
        m = L.shape[0]
        n = L.shape[1]
        assert m == n, "L must be square."
        self.n = n

        def _matvec(x):
            return sp_solve_triangular(self.L, x, trans=0, lower=True)

        def _rmatvec(x):

            return sp_solve_triangular(self.L, x, trans=1, lower=True)
        
        super().__init__( (self.n, self.n), _matvec, _rmatvec, dtype=None, device=device)

    def to_gpu(self):
        raise NotImplementedError
    

In [9]:
# Set the size of the matrix
n = 5

# Generate a random n x n matrix
random_matrix = np.random.rand(n, n)

# Extract the upper triangular part of the matrix
Lmat = np.tril(random_matrix)
L = jlinops.MatrixLinearOperator(Lmat)
Lminv = np.linalg.inv(Lmat)

In [13]:
Linv = jlinops.LowerTriangularInverse(L)

In [14]:
np.linalg.norm( jlinops.black_box_to_dense( Linv ) - Lminv )

1.9513187837406125e-15

In [15]:
np.linalg.norm( jlinops.black_box_to_dense( Linv.T ) - Lminv.T )

2.2763981646657675e-15